In [1]:
!pip install --quiet python-magic langchain llama_index unstructured openai chromadb pdfminer.six pytesseract tiktoken deeplake selenium faiss-cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.34.16 which is incompatible.
datasets 2.12.0 requires dill<0.3.7,>=0.3.0, but you have dill 0.3.7 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.12.2 which is incompatible.


In [1]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [3]:
!pip install -q langchain

In [36]:
print(langchain.__version__)

0.1.7


In [4]:
import pandas as pd
import nltk
import os

import langchain
from langchain.schema import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.document_loaders import OnlinePDFLoader
from langchain.llms import OpenAIChat
from langchain.vectorstores import DeepLake

from langchain.document_loaders import SeleniumURLLoader

from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

ModuleNotFoundError: No module named 'langchain'

In [26]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key=os.getenv("openai_key")

# Reports

In [27]:
df = pd.read_excel('reports.xlsx')

In [28]:
df.shape

(30, 2)

In [29]:
df.head()

,protocol,report
0,Bluzelle,1.1. Main Functions of the Bluzelle and BLZ To...
1,Render,Main Function of the Render Protocol and RNDR ...
2,Loom Network,The Loom Protocol:\nLoom Network facilitates s...
3,Tellor,Main Function of the Tellor Protocol and TRB T...
4,Secret,Main Function of the Secret Protocol and SCRT ...


In [30]:
protocols = df['protocol'].tolist()
reports = df['report'].tolist()

In [31]:
protocols

['Bluzelle',
 'Render',
 'Loom Network',
 'Tellor',
 'Secret',
 'Stacks',
 'Drep',
 'Worldcoin',
 'Conflux',
 'Celer Network',
 'tomiNet',
 'Dogecoin',
 'TRON',
 'Polkadot',
 'Cosmos',
 'Algorand',
 'Optimism',
 'Dai',
 'Monero',
 'Toncoin',
 'Router',
 'MicrovisionChain',
 'SmarDex',
 'FTX ',
 'Shiba Inu',
 'MANA (Decentraland)',
 'Rai Reflex Index (RAI)',
 'Aave ',
 'Maker',
 'Sandbox']

In [32]:
reports[0]

"1.1. Main Functions of the Bluzelle and BLZ Token\n1.1.1 Bluzelle Protocol\nBluzelle is a decentralized data network that aims to allow users to have full control over their data, ensuring both privacy and security. Bluzelle protocol offers a decentralized database service to dApps, aiming to improve data management processes by ensuring data privacy and security through its unique consensus model. The Bluzelle protocol is designed to be interoperable, allowing data to be managed across various blockchain networks.\n1.1.2 BLZ Token\nThe Bluzelle token (BLZ) serves as the native utility token within the ecosystem. BLZ is primarily used to pay for various services within the network, including data storage and retrieval. BLZ token also plays a role in governance, enabling holders to participate in decisions concerning the network's upgrades and changes.\n1.2. Shariah Description of the Bluzelle Protocol and BLZ Token\n1.2.1 The Bluzelle Protocol\nFrom a Shariah perspective, the Bluzelle

In [60]:
# next steps

In [33]:
data = [{'protocol': protocol,'content': report} for protocol,report in zip(protocols,reports)]

In [34]:
data

[{'protocol': 'Bluzelle',
  'content': "1.1. Main Functions of the Bluzelle and BLZ Token\n1.1.1 Bluzelle Protocol\nBluzelle is a decentralized data network that aims to allow users to have full control over their data, ensuring both privacy and security. Bluzelle protocol offers a decentralized database service to dApps, aiming to improve data management processes by ensuring data privacy and security through its unique consensus model. The Bluzelle protocol is designed to be interoperable, allowing data to be managed across various blockchain networks.\n1.1.2 BLZ Token\nThe Bluzelle token (BLZ) serves as the native utility token within the ecosystem. BLZ is primarily used to pay for various services within the network, including data storage and retrieval. BLZ token also plays a role in governance, enabling holders to participate in decisions concerning the network's upgrades and changes.\n1.2. Shariah Description of the Bluzelle Protocol and BLZ Token\n1.2.1 The Bluzelle Protocol\nF

In [35]:
#docs = [Document(title=d['title'], page_content=d['text']) for d in data]
report_docs = [Document(page_content=d['content'],
                metadata={'protocol': d['protocol']}) for d in data]

In [36]:
report_docs

[Document(page_content="1.1. Main Functions of the Bluzelle and BLZ Token\n1.1.1 Bluzelle Protocol\nBluzelle is a decentralized data network that aims to allow users to have full control over their data, ensuring both privacy and security. Bluzelle protocol offers a decentralized database service to dApps, aiming to improve data management processes by ensuring data privacy and security through its unique consensus model. The Bluzelle protocol is designed to be interoperable, allowing data to be managed across various blockchain networks.\n1.1.2 BLZ Token\nThe Bluzelle token (BLZ) serves as the native utility token within the ecosystem. BLZ is primarily used to pay for various services within the network, including data storage and retrieval. BLZ token also plays a role in governance, enabling holders to participate in decisions concerning the network's upgrades and changes.\n1.2. Shariah Description of the Bluzelle Protocol and BLZ Token\n1.2.1 The Bluzelle Protocol\nFrom a Shariah pe

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
#     chunk_size = 1500,
    chunk_overlap  = 200,
    length_function = len,
)

In [45]:
report_docs

[Document(page_content="1.1. Main Functions of the Bluzelle and BLZ Token\n1.1.1 Bluzelle Protocol\nBluzelle is a decentralized data network that aims to allow users to have full control over their data, ensuring both privacy and security. Bluzelle protocol offers a decentralized database service to dApps, aiming to improve data management processes by ensuring data privacy and security through its unique consensus model. The Bluzelle protocol is designed to be interoperable, allowing data to be managed across various blockchain networks.\n1.1.2 BLZ Token\nThe Bluzelle token (BLZ) serves as the native utility token within the ecosystem. BLZ is primarily used to pay for various services within the network, including data storage and retrieval. BLZ token also plays a role in governance, enabling holders to participate in decisions concerning the network's upgrades and changes.\n1.2. Shariah Description of the Bluzelle Protocol and BLZ Token\n1.2.1 The Bluzelle Protocol\nFrom a Shariah pe

In [46]:
report_texts = text_splitter.split_documents(report_docs)

In [48]:
len(report_texts)

32

In [49]:
embeddings = OpenAIEmbeddings(openai_api_key=open_ai_key)

In [50]:
report_db = FAISS.from_documents(report_texts, embeddings)

In [51]:
#docsearch = db
report_db.save_local("faiss_index_report")

# New Code

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key=os.getenv("openai_key")

In [25]:
from docx import Document
import re

def read_docx_and_chunk(file_path):
    document = Document(file_path)
    chunks = []
    current_chunk = []
    prev_heading = "0"

    for paragraph in document.paragraphs:
        text = paragraph.text.strip()
        # Check if the paragraph is a heading (e.g., "1.", "1.1", "2.", "2.1", etc.)
        if re.match(r"\d+(\.\d+)*\s", text):
            heading = text.split()[0]  # Extract the heading number (e.g., "1.", "1.1", etc.)
            if not heading.startswith(prev_heading + ".") and heading.count('.') <= prev_heading.count('.'):
                # If it's a new main section or a sibling of the previous section, save the current chunk and start a new one
                if current_chunk:  # Avoid adding empty chunks
                    chunks.append('\n'.join(current_chunk))
                    current_chunk = []
            prev_heading = heading.rstrip('.')
        current_chunk.append(text)

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append('\n'.join(current_chunk))

    return chunks

file_path = '24 Tokens Review Reports.docx'  # Update this path to your document's location
chunks = read_docx_and_chunk(file_path)
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:\n{chunk}\n{'-'*40}\n")


Chunk 1:
1. Name of the Protocol: ROSE

1.1. Main Function of the Protocol and Token:
The Oasis network platform is a layer one blockchain Proof of Stake smart contract platform that provides scalability, extensibility, and privacy. The main feature of the platform enables efficient verifiable and confidential smart contract execution. It aims to achieve this goal by separating its consensus layer from its layer of contract execution while providing a built-in interface connecting the two for privacy-preserving computation. The consensus layer acts as a hub that uses a Proof-of-Stake (PoS) mechanism to secure the network and reach a consensus on transaction validity. The execution layer consists of multiple, parallel runtimes (called ParaTimes) for specialized computation needs that each plug into the consensus layer.
To access the Oasis Network functions you need to have the ROSE token in your possession. ROSE is used for transaction fees, staking, and delegation at the consensus laye

In [26]:
import langchain
from langchain.schema import Document

In [ ]:
report_docs = [Document(page_content=d['content'],
                metadata={'protocol': d['protocol']}) for d in data]

In [31]:
import langchain
from langchain.schema import Document as LC_Document  # Import LangChain Document as LC_Document to avoid naming conflict
from docx import Document  # This imports Document from python-docx
import re

def read_docx_and_chunk_to_langchain(file_path):
    docx_document = Document(file_path)  # This creates a docx Document object
    langchain_documents = []  # List to hold LangChain Document objects
    current_chunk = []
    prev_heading = "1"

    for paragraph in docx_document.paragraphs:
        text = paragraph.text.strip()
        if re.match(r"\d+(\.\d+)*\s", text):
            heading = text.split()[0]
            if not heading.startswith(prev_heading + ".") and heading.count('.') <= prev_heading.count('.'):
                if current_chunk:  # Save the current chunk as a LangChain Document
                    section_text = '\n'.join(current_chunk)
                    # Create a LangChain Document for the chunk
                    langchain_document = LC_Document(content=section_text, metadata={"heading": prev_heading})
                    langchain_documents.append(langchain_document)
                    current_chunk = []
            prev_heading = heading.rstrip('.')
        current_chunk.append(text)

    # Add the last chunk as a LangChain Document
    if current_chunk:
        section_text = '\n'.join(current_chunk)
        langchain_document = LC_Document(page_content=section_text, metadata={"heading": prev_heading})
        langchain_documents.append(langchain_document)

    return langchain_documents

file_path = '24 Tokens Review Reports.docx'  # Update this path to your document's location
langchain_documents = read_docx_and_chunk_to_langchain(file_path)

# Example: Processing the LangChain documents further or saving them could be done here.



In [32]:
langchain_documents

[Document(page_content='1. Name of the Protocol: ROSE\n\n1.1. Main Function of the Protocol and Token:\nThe Oasis network platform is a layer one blockchain Proof of Stake smart contract platform that provides scalability, extensibility, and privacy. The main feature of the platform enables efficient verifiable and confidential smart contract execution. It aims to achieve this goal by separating its consensus layer from its layer of contract execution while providing a built-in interface connecting the two for privacy-preserving computation. The consensus layer acts as a hub that uses a Proof-of-Stake (PoS) mechanism to secure the network and reach a consensus on transaction validity. The execution layer consists of multiple, parallel runtimes (called ParaTimes) for specialized computation needs that each plug into the consensus layer.\nTo access the Oasis Network functions you need to have the ROSE token in your possession. ROSE is used for transaction fees, staking, and delegation at

In [33]:
len(langchain_documents)

1

In [35]:
import langchain
from langchain.schema import Document as LC_Document
from docx import Document
import re

def read_docx_and_compile_to_single_langchain_document(file_path):
    docx_document = Document(file_path)
    sections = []  # List to hold sections as dictionaries
    current_chunk = []
    prev_heading = "0"

    for paragraph in docx_document.paragraphs:
        text = paragraph.text.strip()
        if re.match(r"\d+(\.\d+)*\s", text):
            heading = text.split()[0]
            if not heading.startswith(prev_heading + ".") and heading.count('.') <= prev_heading.count('.'):
                if current_chunk:  # Save the current chunk as a section
                    section_text = '\n'.join(current_chunk)
                    # Append the chunk as a section dictionary
                    sections.append({"content": section_text, "metadata": {"heading": prev_heading}})
                    current_chunk = []
            prev_heading = heading.rstrip('.')
        current_chunk.append(text)

    # Add the last chunk as a section
    if current_chunk:
        section_text = '\n'.join(current_chunk)
        sections.append({"content": section_text, "metadata": {"heading": prev_heading}})

    # Create a single LangChain Document encompassing all sections
    single_langchain_document = LC_Document(page_content=sections, metadata={"document": "Compiled from chunks"})

    return single_langchain_document

file_path = '24 Tokens Review Reports.docx'  # Update this path to your document's location
compiled_document = read_docx_and_compile_to_single_langchain_document(file_path)

# Now, `compiled_document` is a single LangChain Document containing all sections.


ValidationError: 1 validation error for Document
page_content
  str type expected (type=type_error.str)